# Embedded CFG Experiment `timer`

Testing the embedded FXE implementation on `timer` example binaries from the nRF5 SDK (v17.0.2)

In [1]:
import os
from copy import deepcopy

from efxe import *

In [2]:
%%time
timer_efxe = EmFXEngine(
    pd="mmaps/nrf52832.yml",
    fw_path="examples/timer.elf",
    log_stdout=True,
    log_insn=True,
)
timer_efxe.run()

2022-11-20 10:53:13,134 : backup context @ 0x2b1
2022-11-20 10:53:13,136 : backup context @ 0x2b1
2022-11-20 10:53:13,136 : backup context @ 0x2b1
2022-11-20 10:53:13,137 : backup context @ 0x2b1
2022-11-20 10:53:13,137 : backup context @ 0x2b1
2022-11-20 10:53:13,137 : backup context @ 0x2b1
2022-11-20 10:53:13,138 : backup context @ 0x2b1
2022-11-20 10:53:13,138 : backup context @ 0x2b1
2022-11-20 10:53:13,138 : backup context @ 0x2b1
2022-11-20 10:53:13,139 : backup context @ 0x2b1
2022-11-20 10:53:13,139 : backup context @ 0x2b1
2022-11-20 10:53:13,139 : backup context @ 0x2b1
2022-11-20 10:53:13,140 : backup context @ 0x2b1
2022-11-20 10:53:13,140 : backup context @ 0x2b1
2022-11-20 10:53:13,141 : backup context @ 0x2b1
2022-11-20 10:53:13,141 : backup context @ 0x2b1
2022-11-20 10:53:13,141 : backup context @ 0x2b1
2022-11-20 10:53:13,142 : backup context @ 0x2b1
2022-11-20 10:53:13,142 : backup context @ 0x2b1
2022-11-20 10:53:13,142 : backup context @ 0x2b1
2022-11-20 10:53:13,

UcError: Invalid memory write (UC_ERR_WRITE_UNMAPPED)

In [3]:
total_insns = 0
total_size = 0
for block in timer_efxe.bblocks.values():
    total_size += block.size
    for insn in timer_efxe.cs.disasm(
            timer_efxe.uc.mem_read(block.addr, block.size), block.size):
        total_insns += 1
print(f"Found {len(timer_efxe.bblocks)} blocks, {total_insns} insns, {hex(total_size)} bytes\n")

Found 206 blocks, 979 insns, 0xaa0 bytes



In [4]:
disasm_txt = deepcopy(timer_efxe.fw.disasm_txt)
for block in timer_efxe.bblocks.values():
    insn_addr = block.addr
    for insn in timer_efxe.cs.disasm(
            timer_efxe.uc.mem_read(block.addr, block.size), block.size):
        try:
            lineno = timer_efxe.fw.disasm[insn_addr]['line']
            insn_addr += insn.size
            disasm_txt[lineno] = "{}{}{}{:<100}\u001b[0m".format(
                "\u001b[103m",  # bright yellow background
                "\u001b[30m",   # black foreground
                "\u001b[1m",    # bold font
                disasm_txt[lineno])
        except Exception:
            pass
print('\n'.join(disasm_txt))


examples/timer.elf:     file format elf32-littlearm


Disassembly of section .text:

00000000 <__isr_vector>:
   0:	20010000 	.word	0x20010000
   4:	000002b1 	.word	0x000002b1
   8:	000002d9 	.word	0x000002d9
   c:	000002db 	.word	0x000002db
  10:	000002dd 	.word	0x000002dd
  14:	000002df 	.word	0x000002df
  18:	000002e1 	.word	0x000002e1
	...
  2c:	000002e3 	.word	0x000002e3
  30:	000002e5 	.word	0x000002e5
  34:	00000000 	.word	0x00000000
  38:	000002e7 	.word	0x000002e7
  3c:	000002e9 	.word	0x000002e9
  40:	000002eb 	.word	0x000002eb
  44:	000002eb 	.word	0x000002eb
  48:	000002eb 	.word	0x000002eb
  4c:	000002eb 	.word	0x000002eb
  50:	000002eb 	.word	0x000002eb
  54:	000002eb 	.word	0x000002eb
  58:	000002eb 	.word	0x000002eb
  5c:	000002eb 	.word	0x000002eb
  60:	00000541 	.word	0x00000541
  64:	000002eb 	.word	0x000002eb
  68:	000002eb 	.word	0x000002eb
  6c:	000002eb 	.word	0x000002eb
  70:	000002eb 	.word	0x000002eb
  74:	000002eb 	.word	0x000002eb
  78:	000002eb 	.word	0x00